In [194]:
import pandas as pd
import numpy as np
import pickle
import spacy
from spacy.lang.de import German
from spacy.matcher import Matcher
from spacy.tokens import Span, Token, Doc
from geopy.geocoders import Nominatim
import regex as re

# Load pickled df_lgbt

In [2]:
with open('pickles/df_lgbt.pickle', 'rb') as f:
    df_lgbt = pickle.load(f)

# Load pickled df_map (test)

In [241]:
with open('pickles/df_map_test.pickle', 'rb') as f:
    df_map = pickle.load(f)

# Load SpaCy docs

In [7]:
with open('pickles/spacy_docs.pickle', 'rb') as f:
    docs = pickle.load(f)


**Add custom attributes 'index' and 'district' to Doc**

In [8]:
Doc.set_extension('index', default=None)
for doc, context in docs:
    doc._.index = context["index"]
docs[0][0]._.index

1707

In [9]:
Doc.set_extension('district', default=None)
for doc, context in docs:
    doc._.district = context["district"]
docs[0][0]._.district

'Tempelhof-Schöneberg'

____

## Find locations

In [4]:
# load pickled dict_no_ents:
with open('pickles/dict_no_ents.pickle', 'rb') as f:
    dict_no_ents = pickle.load(f)

In [49]:
dict_no_ents[1861]

{'location': 'Denkmal für die im Nationalsozialismus verfolgten Homosexuellen',
 'attack': None,
 'propaganda': 'anonymous',
 'damage_of_property': 'yes',
 'structural_discrimination': None,
 'reported_police': 'yes'}

In [135]:
Nominatim(user_agent="mymap").geocode('Franz-Marc-Schule Berlin', addressdetails=True).raw

{'place_id': 77623864,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'osm_type': 'node',
 'osm_id': 7133350405,
 'boundingbox': ['52.5881058', '52.5882058', '13.2826944', '13.2827944'],
 'lat': '52.5881558',
 'lon': '13.2827444',
 'display_name': 'Franz-Marc-Grundschule, 26-31, Treskowstraße, Tegel, Reinickendorf, Berlin, 13507, Deutschland',
 'class': 'amenity',
 'type': 'school',
 'importance': 0.411,
 'icon': 'https://nominatim.openstreetmap.org/ui/mapicons//education_school.p.20.png',
 'address': {'amenity': 'Franz-Marc-Grundschule',
  'house_number': '26-31',
  'road': 'Treskowstraße',
  'suburb': 'Tegel',
  'borough': 'Reinickendorf',
  'city': 'Berlin',
  'state': 'Berlin',
  'postcode': '13507',
  'country': 'Deutschland',
  'country_code': 'de'}}

In [61]:
doc = [doc for doc in docs if doc[0]._.index == 1861][0]

In [145]:
df_map = pd.concat([df_map, df_map_new])

In [161]:
df_map.loc[3403, 'Story']

'Am 29. September 2015 ereignete sich im Ortsteil Mitte ein Angriff aufgrund von Homophobie, zu dem die Berliner Polizei aus datenschutzrechtlichen Gründen keine näheren Angaben machte.'

In [183]:
df_map.loc[:,'Attack'].fillna('check', inplace=True)

In [224]:
df_map.loc[geschlagen_list, 'Attack'] = 'physical'

In [233]:
df_map[(df_map['Header'].str.contains('geschlagen') == True) & (df_map['Attack'] != 'physical')]

,Date,District,Header,Story,Source,Attack,latitude,longitude
2350,2014-05-18,Friedrichshain-Kreuzberg,Mann aus homophoben Gründen zu Boden geschlagen,,ReachOut,other,52.516113,13.448219


In [236]:
df_lgbt.loc[2350, 'Attack'] = 'physical'

In [211]:
df_lgbt[df_lgbt['Story'].str.contains('BVV') == True]

,Date,District,Header,Story,Source,Attack,Propaganda,Structural Discrimination,Material Damage,Reported status
6609,2016-04-14,Charlottenburg-Wilmersdorf,Bezirkswahlprogramm der AfD,Am 14. April 2016 verabschiedete die Partei Al...,Register CW,NaN,NaN,NaN,NaN,NaN
7517,2017-12-14,Lichtenberg,AfD-Anträge in der Lichtenberger BVV,Mit einem Antrag in der BVV versuchte die Lich...,Lichtenberger Register,verbal,NaN,NaN,NaN,NaN
7702,2017-11-16,Lichtenberg,Diskriminierende Wortmeldungen der AfD in der BVV,Während der Lichtenberger BVV meldete sich die...,Lichtenberger Register,NaN,NaN,NaN,NaN,NaN
7976,2017-09-30,Treptow-Köpenick,"Facebook-Dokumentation September: ""AfD Treptow...",Der September war vom Wahlkampf geprägt. 15 de...,Register Treptow-Köpenick,NaN,NaN,NaN,NaN,NaN
8645,2017-07-31,Treptow-Köpenick,"Facebook-Dokumentation Juli: ""AfD Treptow-Köpe...",Im Juli setzte sich der Wahlkampf auf der Seit...,Register Treptow-Köpenick,NaN,NaN,NaN,NaN,NaN
8796,2017-07-13,Lichtenberg,AfD-Anträge in der Lichtenberger BVV,Die Lichtenberger AfD stellte während der BVV ...,Lichtenberger Register,NaN,NaN,NaN,NaN,NaN
8895,2017-06-30,Treptow-Köpenick,"Facebook-Dokumentation Juni: ""AfD Treptow-Köpe...",Im Juni startete die AfD in den Wahlkampf. In ...,Register Treptow-Köpenick,NaN,NaN,NaN,NaN,NaN
9443,2017-04-28,Friedrichshain-Kreuzberg,"Mail mit rassistischem, fremdenfeindlichen und...",Eine AfD-Bezirksverordnete beschrieb in einen ...,Register Friedrichshain-Kreuzberg,NaN,NaN,NaN,NaN,NaN
10621,2018-11-14,Neukölln,BVV Neukölln: Diffamierung politischer Gegner*...,"In ihren Beiträgen versuchte die AfD, die Arbe...","Register Neukölln, Mobile Beratung gegen Recht...",physical,NaN,NaN,NaN,NaN
12142,2018-06-14,Treptow-Köpenick,Diffamierung der Arbeit von Jugendfreizeiteinr...,Der Antrag der SPD und den Grünen „Fachtag zu ...,Register Treptow-Köpenick,NaN,NaN,NaN,NaN,NaN


In [185]:
df_map[df_map['Attack'].isna() == True]

,Date,District,Header,Story,Source,Attack,latitude,longitude


**BVV**

In [191]:
docs[0][0]._.index

1707

In [196]:
BVV_indices = []
BVV_findings = []
for doc in docs:
    found_list = re.findall('BVV',doc[0].text)
    if len(BVV_findings) > 0:
        BVV_indices.append(doc[0]._.index)
        BVV_findings.append(found_list)

In [206]:
len(re.findall('BVV',docs[0][0].text))

0

In [207]:
df_lgbt

,Date,District,Header,Story,Source,Attack,Propaganda,Structural Discrimination,Material Damage,Reported status
1707,2014-11-29,Tempelhof-Schöneberg,Angriff aus transphober Motivation,Ein 30-jähriger Mann wird gegen 05.00 Uhr in d...,ReachOut,physical,NaN,NaN,NaN,NaN
1836,2014-10-26,Tempelhof-Schöneberg,Angriff aus transphober Motivation,Am 26.10.14 wird ein 30-jähriger Mann gegen 07...,ReachOut,physical,NaN,NaN,NaN,NaN
1861,2014-10-17,Mitte,LGBQT feindliche Schmiererei entdeckt!,Am Abend gegen 20.10 Uhr stellen Polizeibeamte...,Polizei Berlin,NaN,NaN,NaN,NaN,NaN
1874,2014-10-12,Marzahn-Hellersdorf,Menschenfeindliche Parolen beim Sport,Gegen 16:00 Uhr werden Fans des Fußballvereins...,Register Charlottenburg-Wilmersdorf; Antisemit...,verbal,NaN,NaN,NaN,NaN
1913,2014-10-03,Charlottenburg-Wilmersdorf,Homosexuellenfeindliches Banner bei Stuttgarte...,Am 3. Oktober 2014 hing im Olympiastadion in W...,"Fußballfans gegen Homophobie, Hertha-Inside.de",NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
21900,2021-01-14,Friedrichshain-Kreuzberg,Homophobe Beleidigung während Online-Unterrichts,Ein Lehrer einer Kreuzberger Schule wurde anon...,Bürger*innenmeldung,verbal,NaN,NaN,NaN,NaN
21930,2021-01-10,Treptow-Köpenick,Extrem rechte Propaganda des 3. Weg in Johanni...,"Im Gebiet zwischen Landschaftspark, Gerhard-Se...",Zentrum für Demokratie,NaN,NaN,NaN,NaN,NaN
21933,2021-01-10,Lichtenberg,NS-Propaganda am Fennpfuhl,"Am Anton-Saefkow-Platz, in der Bernhard-Bästle...",Berliner Register,NaN,NaN,NaN,NaN,NaN
21948,2021-01-07,Mitte,Homophobe Beleidigung und Bedrohung im U-Bahnh...,Im U-Bahnhof Seestraße wurde gegen 8.20 Uhr ei...,"Registerstelle Mitte, Twitter",verbal,NaN,NaN,NaN,NaN


In [240]:
df_map.loc[1861]

Date                                       2014-10-17 00:00:00
District                                                 Mitte
Header                  LGBQT feindliche Schmiererei entdeckt!
Story        Am Abend gegen 20.10 Uhr stellen Polizeibeamte...
Source                                          Polizei Berlin
Attack                                                   check
latitude                                               52.5133
longitude                                              13.3761
Name: 1861, dtype: object

In [136]:
for (k,v) in dict_no_ents.items():
    try:
        assert type(v['location']) == str
    except:
        continue
    doc = [doc for doc in docs if doc[0]._.index == k][0]
    district = doc[0]._.district
    loc_string = v['location']
    loc_raw = Nominatim(user_agent="mymap").geocode(loc_string + ' Berlin', addressdetails=True).raw
    lat = float(loc_raw['lat'])
    lon = float(loc_raw['lon'])
    df_map_new.loc[k, 'latitude'] = lat
    df_map_new.loc[k, 'longitude'] = lon

In [132]:
doc

(Rassistische und homophobe Schmiererei. Vor der Franz-Marc-Schule in der Treskowallee ist auf einem Papierkorb mit schwarzem Edding "Erdogan = schwul" angebracht. Dieser Slogan tauchte in der Gegend bereits öfter und in Verbindung mit anderen rassistischen Sprüchen auf.,
 {'index': 6214,
  'district': 'Reinickendorf',
  'ORG': ['Franz-Marc-Schule'],
  'LOC': ['Treskowallee'],
  'PER': ['Erdogan']})

In [133]:
dict_no_ents[6214]

{'location': 'Franz-Marc-Schule',
 'attack': None,
 'propaganda': 'anonymous',
 'damage_of_property': 'yes',
 'structural_discrimination': None,
 'reported_police': None}

In [130]:
dict_no_ents[5421]['location'] = 'U Hönow'

In [27]:
df_map_new = df_lgbt.loc[dict_no_ents.keys(), ['Date','District','Header','Story','Source','Attack']]

In [37]:
df_map_new[df_map_new['Attack'].isna() == True]


,Date,District,Header,Story,Source,Attack
1861,2014-10-17,Mitte,LGBQT feindliche Schmiererei entdeckt!,Am Abend gegen 20.10 Uhr stellen Polizeibeamte...,Polizei Berlin,NaN
3206,2015-10-30,Friedrichshain-Kreuzberg,Brief mit antisemitischem und homophobem Inhalt,Im Axel-Springer-Verlag ging ein Brief mit ant...,Polizei,NaN
3645,2015-08-23,Mitte,Pfefferspray gegen Homosexuellen,Am 23. August griff eine Gruppe von Männern ei...,ReachOut e.V.,NaN
3763,2015-07-31,Lichtenberg,"Hetze von ""Pro Deutschland"" auf Facebook",Die Kleinstpartei „Pro Deutschland“ veröffentl...,Lichtenberger Register,NaN
4032,2015-06-06,Friedrichshain-Kreuzberg,Mann aus homophober Motivation ins Gesicht ges...,Ein 31-jähriger Mann wird gegen 0.20 Uhr in de...,ReachOut,NaN
4405,2015-03-23,Treptow-Köpenick,Homophobe Vermietungspolitik,Zwei Frauen versuchten bei einer privaten Gart...,Zentrum für Demokratie,NaN
5183,2016-10-13,Friedrichshain-Kreuzberg,LGBTIQ*-feindliche Schmiererei am U-Bahnhof Mö...,Am U-Bahnhof Möckernbrücke wurde eine LGBTIQ* ...,Register Charlottenburg-Wilmersdorf,NaN
5830,2016-07-23,Mitte,Homophob angegriffen,Gegen 16.00 Uhr beobachtet ein 49-jähriger Man...,"Polizei Berlin, 27.02.2017, ReachOut",NaN
6214,2016-06-03,Reinickendorf,Rassistische und homophobe Schmiererei,Vor der Franz-Marc-Schule in der Treskowallee ...,Register Spandau,NaN
6221,2016-06-02,Reinickendorf,Plakate beschmiert und beschädigt,In der ver.di Jugendbildungsstätte Konradshöhe...,ver.di Jugendbildungsstätte Berlin-Konradshöhe...,NaN


In [5]:
df_map

,Date,District,Header,Story,Source,Attack,latitude,longitude
1707,2014-11-29,Tempelhof-Schöneberg,Angriff aus transphober Motivation,Ein 30-jähriger Mann wird gegen 05.00 Uhr in d...,ReachOut,physical,52.499447,13.359882
1836,2014-10-26,Tempelhof-Schöneberg,Angriff aus transphober Motivation,Am 26.10.14 wird ein 30-jähriger Mann gegen 07...,ReachOut,physical,52.494660,13.361087
1915,2014-10-03,Mitte,Homophobe Aggression gegen ein schwules Pärchen,Zwei Schwule werden auf dem U-Bahnhof Pankstra...,Polizei Berlin,other,52.550920,13.384846
1940,2014-09-24,Tempelhof-Schöneberg,Angriff aus transphober Motivation,Auf der Potsdamer Straße ist gegen 11.20 Uhr e...,ReachOut,physical,52.494660,13.361087
1951,2014-09-21,Pankow,Homophober Angriff am S-Bahnhof Blankenburg,Zwei 20- und 21-jährige Männer werden gegen 00...,ReachOut,physical,52.591013,13.442641
...,...,...,...,...,...,...,...,...
21633,2021-02-15,Friedrichshain-Kreuzberg,LGBTI*-feindliche Sticker im Boxikiez,In der Kreutziger Ecke Boxhagener Straße sowie...,Register Friedrichshain-Kreuzberg,other,52.510799,13.459701
21649,2021-02-13,Neukölln,Extrem rechtes Plakat bei S-Hermannstr.,Eine extrem rechte Gruppierung tauschte ein We...,"Register Neukölln, via twitter",other,52.467634,13.431241
21745,2021-02-02,Tempelhof-Schöneberg,Kondolenzkranz am U-Bhf. Nollendorfplatz zerstört,Ein Betrunkener zerstörte am Eingang des U-Bah...,"Tagesspiegel, 2.2.21",other,52.499538,13.353943
21858,2021-01-20,Treptow-Köpenick,Hakenkreuz u.a. in Friedrichshagen,An einer Schallschutzmauer am S-Bhf. Friedrich...,Zentrum für Demokratie,other,52.449380,13.626272


In [165]:
df_lgbt.head()

,Date,District,Header,Story,Source,Attack,Propaganda,Structural Discrimination,Material Damage,Reported status
1707,2014-11-29,Tempelhof-Schöneberg,Angriff aus transphober Motivation,Ein 30-jähriger Mann wird gegen 05.00 Uhr in d...,ReachOut,physical,NaN,NaN,NaN,NaN
1836,2014-10-26,Tempelhof-Schöneberg,Angriff aus transphober Motivation,Am 26.10.14 wird ein 30-jähriger Mann gegen 07...,ReachOut,physical,NaN,NaN,NaN,NaN
1861,2014-10-17,Mitte,LGBQT feindliche Schmiererei entdeckt!,Am Abend gegen 20.10 Uhr stellen Polizeibeamte...,Polizei Berlin,NaN,NaN,NaN,NaN,NaN
1874,2014-10-12,Marzahn-Hellersdorf,Menschenfeindliche Parolen beim Sport,Gegen 16:00 Uhr werden Fans des Fußballvereins...,Register Charlottenburg-Wilmersdorf; Antisemit...,verbal,NaN,NaN,NaN,NaN
1913,2014-10-03,Charlottenburg-Wilmersdorf,Homosexuellenfeindliches Banner bei Stuttgarte...,Am 3. Oktober 2014 hing im Olympiastadion in W...,"Fußballfans gegen Homophobie, Hertha-Inside.de",NaN,NaN,NaN,NaN,NaN


In [117]:
docs[859]

(AfD-Abgeordneter hetzt gegen LGBTIQ*-Parade. Ein AfD-Abgeordneter aus Marzahn-Hellersdorf hat die Teilnehmenden einer LGBTIQ*-Parade am Viktor-Klemperer-Platz gefilmt und anschließend in einem Video im Internet diffamiert. Er behauptet in dem Video, die Teilnehmenden kämen zu großen Teilen von außerhalb und würden "Unruhe stiften" und "Krawall machen". Weiterhin behauptet er, die Russlanddeutsche Community in Berlin lebe per se nach einem wertkonservativen Weltbild, weshalb die Parade einer vermeintlichen "Umerziehung" dienen würde. Weiterhin behauptet er, dies sei das Ziel von "Rot-Rot-Grün". Bereits am Tag zuvor behauptete er in seinen Social Media Beiträgen die LGBTIQ*-Parade, die von einem russischsprachigen Verein organisiert wird, sei "antirussischer Rassismus". Mit seinen Aussagen spricht der AfD-Abgeordnete dem russland-deutschen Leben in Berlin die Vereinbarkeit mit diversen Lebensweisen ab und schürt gleichzeitig Feindlichkeit gegen diese, indem er behauptet, diese würden de

In [150]:
list(docs[55][1].values())[2:]

[['Homophob'],
 ['Prenzlauer Berg', 'Bornholmer Straße', 'Pankow'],
 ['Polizeiliche Staatschutz']]

In [ ]:
# TODO
'''
If it finds an unambiguos location: adds three entries to the doc_dict of a doc: 
    - location, latitude, longitude
Saves selected raw Nominatim location object in a dictionary where the keys are the indices
If location is ambiguous, saves the story index in a list

This function should call a desambiguator when location is not a specific address
The function iterates through the entities and select those that can be found in the given district.

Returns
    -dict of Nominatim locations
    -list of ambiguous locations


'''

In [166]:
Nominatim(user_agent="mymap").geocode('ficken berlin', addressdetails=True).raw

GeocoderUnavailable: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=ficken+berlin&format=json&limit=1&addressdetails=1 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x143886730>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))

In [ ]:
def location_getter(doc):
    '''
    Adds key to doc_dict with list of Nominatim locations found in the district of the incident
    '''
    try: # maybe try is redundant
        for ent_list in list(doc[1].values())[2:]:
            for ent in ent_list:    
                query = ent + ' ' + district + ' Berlin'
                loc = Nominatim(user_agent="mymap").geocode(query, addressdetails=True).raw
                if loc['address']['borough'] == district:
                    if len(doc[1]['location']) == 0:
                        doc[1]['locations'] = [loc]
                    else:
                        doc[1]['locations'].append(loc)
                    
        try:
            loc = Nominatim(user_agent="mymap").geocode(query, addressdetails=True).raw
            if loc['address']['borough'] == district:
                good_loc_list.append(loc)
            # query found location but not the in the expected district
            elif loc['address']['borough'] != district:
                bad_loc_list.append(loc)
        # query didn't find any location
        except:
            bad_ents.append(ent)
            
    return good_loc_list, bad_loc_list, bad_ents_list

In [116]:
docs[476]

(Aus LGBTIQ-feindlicher Motivation getreten in Nordneukölln. Gegen 21.15 Uhr war ein 28-Jähriger in Begleitung seines gleichaltrigen Ehemanns unterwegs. In der Richardstraße, Ecke Kirchgasse wurde der 28-Jährige von einem 16-jährigen Jugendlichen aus LGBTIQ-feindlicher Motivation beleidigt, bedroht, getreten und verletzt. Ein unbekannter Begleiter des 16-Jährigen Angreifers schritt ein und verhinderte weitere Attacken.
 Die Polizei wurde verständigt und nahm den Tatverdächtigen fest.,
 {'index': 12434,
  'district': 'Neukölln',
  'ORG': ['Nordneukölln'],
  'LOC': ['Richardstraße'],
  'MISC': ['LGBTIQ-feindlicher']})

In [376]:
Nominatim(user_agent="mymap").geocode("Denkmal für die erste homosexuelle Emanzipationsbewegung berlin", addressdetails=True).raw

{'place_id': 57919389,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'osm_type': 'node',
 'osm_id': 4944467707,
 'boundingbox': ['52.5184532', '52.5185532', '13.3599952', '13.3600952'],
 'lat': '52.5185032',
 'lon': '13.3600452',
 'display_name': 'Denkmal für die erste homosexuelle Emanzipationsbewegung, Magnus-Hirschfeld-Ufer, Moabit, Mitte, Berlin, 10557, Deutschland',
 'class': 'historic',
 'type': 'memorial',
 'importance': 0.7010000000000001,
 'icon': 'https://nominatim.openstreetmap.org/ui/mapicons//tourist_monument.p.20.png',
 'address': {'historic': 'Denkmal für die erste homosexuelle Emanzipationsbewegung',
  'road': 'Magnus-Hirschfeld-Ufer',
  'suburb': 'Moabit',
  'borough': 'Mitte',
  'city': 'Berlin',
  'state': 'Berlin',
  'postcode': '10557',
  'country': 'Deutschland',
  'country_code': 'de'}}

In [39]:
np.random.randint(1, 50)

14

In [44]:
def desambiguate_str(doc):
    '''
    takes spacy doc
    returns tuple (lat, lon)
    '''
    district = doc[0]._.district
    address_str = "Frobenstraße" + " " + str(np.random.randint(1, 500)) + " " + district + " Berlin"
Nominatim(user_agent="mymap").geocode("Frobenstraße","Tempelhof-Schöneberg",
    str(np.random.randint(1, 50)),
    "Berlin",
    addressdetails=True
).raw

TypeError: geocode() takes 2 positional arguments but 5 positional arguments (and 1 keyword-only argument) were given

**Find coordinates**

In [ ]:
# other location finders
# geopandas geojsaon

In [6]:
def location_finder(ents, district):
    '''
    Returns a list of raw Nominatim location objects, which include methods to retrieve coordinates.
    The function iterates through the entities and select those that can be found in the given district.
    
    ents:
        tuple of SpaCy entities in list_of_entities = [doc[0].ents for doc in docs]
    distric:
        string with the district name
        
    '''
    good_loc_list = []
    bad_loc_list = []
    bad_ents_list = []
    for ent in ents:
        query = ent.text, district,'Berlin'
        try:
            loc = Nominatim(user_agent="mymap").geocode(query, addressdetails=True).raw
            if loc['address']['borough'] == district:
                good_loc_list.append(loc)
            # query found location but not the in the expected district
            elif loc['address']['borough'] != district:
                bad_loc_list.append(loc)
        # query didn't find any location
        except:
            bad_ents.append(ent)
            
    return good_loc_list, bad_loc_list, bad_ents_list

In [134]:
list_of_entities = [doc[0].ents for doc in docs]
list_of_districts = list(df_lgbt['District'])

In [15]:
df_map[['latitude','longitude']].value_counts()

latitude   longitude
52.464647  13.473014    13
52.463850  13.443969     9
52.530102  13.342542     8
52.515747  13.309683     7
52.497644  13.411914     6
                        ..
52.514708  13.398153     1
52.514334  13.460864     1
52.513739  13.337984     1
52.513633  13.440270     1
52.408821  13.267382     1
Length: 224, dtype: int64

In [ ]:
# desambiguate repeated coordinates
#[(lat,lon) for lat, lon in zip(df_map_test['latitude'], df_map_test['longitude'])]
# Nominatim(user_agent="mymap").geocode('hermannstr berlin ', addressdetails=True)
# Nominatim(user_agent="mymap").geocode('berghain berlin ', addressdetails=True)

def street_desambiguator(list_of_docs):
    '''
    Takes a list of SpaCy docs in the same street location and randomly assigns a house number
    so they have different addresses
    Returns 
    '''
    for doc in list_of_docs:
        doc.

# Solve missing values df_map

In [385]:
df_map.index

Int64Index([ 1707,  1836,  1915,  1940,  1951,  1974,  1985,  2126,  2151,
             2215,
            ...
            18842, 19048, 19152, 19203, 19354, 19591, 19700, 20279, 20746,
            21948],
           dtype='int64', length=429)

In [348]:
denkmal_indices = np.array(df_lgbt[df_lgbt['Story'].str.contains('Denkmal') == True].index)

In [388]:

denkmal_indices_mitte = [ 10437, 10926, 14025, 14548, 14745, 14858, 15174, 15355, 16735, 19214]
denkmal_indices_moabit = [4133, 14383] 

In [390]:
[i for i in list(df_map.index) if i in denkmal_indices_mitte]

[]

In [409]:
df_map.head()

,Date,District,Header,Story,Source,Attack,latitude,longitude,Propaganda,Structural Discrimination,Material Damage,Reported status
1707,2014-11-29,Tempelhof-Schöneberg,Angriff aus transphober Motivation,Ein 30-jähriger Mann wird gegen 05.00 Uhr in d...,ReachOut,physical,52.4994,13.3599,Unknown,Unknown,Unknown,Unknown
1836,2014-10-26,Tempelhof-Schöneberg,Angriff aus transphober Motivation,Am 26.10.14 wird ein 30-jähriger Mann gegen 07...,ReachOut,physical,52.4947,13.3611,Unknown,Unknown,Unknown,Unknown
1915,2014-10-03,Mitte,Homophobe Aggression gegen ein schwules Pärchen,Zwei Schwule werden auf dem U-Bahnhof Pankstra...,Polizei Berlin,physical,52.5509,13.3848,Unknown,Unknown,Unknown,Unknown
1940,2014-09-24,Tempelhof-Schöneberg,Angriff aus transphober Motivation,Auf der Potsdamer Straße ist gegen 11.20 Uhr e...,ReachOut,physical,52.4947,13.3611,Unknown,Unknown,Unknown,Unknown
1951,2014-09-21,Pankow,Homophober Angriff am S-Bahnhof Blankenburg,Zwei 20- und 21-jährige Männer werden gegen 00...,ReachOut,physical,52.591,13.4426,Unknown,Unknown,Unknown,Unknown


In [420]:
df_denkmal_new.loc[denkmal_indices_moabit, 'latitude'] = 52.5185032

In [422]:
len(df_map)

429

In [425]:
df_map = pd.concat([df_map, df_denkmal_new])

In [396]:
df_denkmal_new = df_lgbt.loc[denkmal_indices_mitte + denkmal_indices_moabit ]


In [415]:
df_map[df_map['Story'].str.contains('Denkmal') == True]['longitude']

1861    13.3761
Name: longitude, dtype: object

In [374]:
# 1, 5 [4133, 14383] go to '52.5185032', lon': '13.3600452'

[doc[0] for doc in docs if doc[0]._.index in denkmal_indices]

[LGBQT feindliche Schmiererei entdeckt!. Am Abend gegen 20.10 Uhr stellen Polizeibeamte eine Farbschmiererei an dem Denkmal für die im Nationalsozialismus verfolgten Homosexuellen fest.,
 Schmierereien an den Gedenktafeln am Magnus-Hirschfeld-Ufer. Am Magnus-Hirschfeld-Ufer gegenüber dem Bundeskanzleramt wurden erneut die beiden Gedenktafeln, mit denen an die weltweit erste homosexuelle Emanzipationsbewegung erinnert wird, beschädigt. Die Tafeln wurden auf der Vorder- und Rückseite mit Schriftzeichen und Symbolen beschmiert – darunter auch ein Penis. Der Lesben- und Schwulenverband Berlin-Brandenburg (LSVD) hat Anzeige gegen Unbekannt gestellt und den Berliner Senat um schnellstmögliche Schadensbehebung gebeten.
 Bereits im vergangenen Jahr haben sich in Berlin die Sachbeschädigungen an Erinnerungsorten der homosexuellen Verfolgungsgeschichte sowie der homosexuellen Emanzipationsbewegung gehäuft. Hierzu zählen unter anderem die Beschädigung der Gedenktafel für die homosexuellen Opfer d

In [336]:
df_map.loc[[7517, 13208, 16069],'Attack'] = 'No'

In [339]:
df_map.loc[[7517, 7702, 7976, 12142, 13208, 16069]]#,'Propaganda'] = 'yes'

,Date,District,Header,Story,Source,Attack,latitude,longitude,Propaganda,Structural Discrimination,Material Damage,Reported status
7517,2017-12-14,Lichtenberg,AfD-Anträge in der Lichtenberger BVV,Mit einem Antrag in der BVV versuchte die Lich...,Lichtenberger Register,No,52.5097,13.5036,yes,Unknown,Unknown,Unknown
7702,2017-11-16,Lichtenberg,Diskriminierende Wortmeldungen der AfD in der BVV,Während der Lichtenberger BVV meldete sich die...,Lichtenberger Register,other,52.5097,13.5036,yes,Unknown,Unknown,Unknown
7976,2017-09-30,Treptow-Köpenick,"Facebook-Dokumentation September: ""AfD Treptow...",Der September war vom Wahlkampf geprägt. 15 de...,Register Treptow-Köpenick,other,52.4437,13.5641,yes,Unknown,Unknown,Unknown
12142,2018-06-14,Treptow-Köpenick,Diffamierung der Arbeit von Jugendfreizeiteinr...,Der Antrag der SPD und den Grünen „Fachtag zu ...,Register Treptow-Köpenick,other,52.4499,13.5124,yes,Unknown,Unknown,Unknown
13208,2018-03-01,Treptow-Köpenick,Rassistische Feindkonstruktion und Bedrohung i...,In der BVV stellte die AfD einen Änderungsantr...,Register Treptow-Köpenick,No,52.4499,13.5124,yes,Unknown,Unknown,Unknown
16069,2019-04-10,Neukölln,Rassistische Wortmeldungen in BVV,Während der Neuköllner BVV-Sitzung am 10. Apri...,Register Neukölln,No,52.4811,13.4354,yes,Unknown,Unknown,Unknown


In [306]:
df_map[['Attack']].value_counts()

Attack  
physical    241
other        93
verbal       87
No            9
dtype: int64

In [303]:
df_map.loc[[3763, 8212, 8891], 'Propaganda'] = 'yes'

In [304]:
df_map[df_map['Propaganda'] != 'Unknown']

,Date,District,Header,Story,Source,Attack,latitude,longitude,Propaganda,Structural Discrimination,Material Damage,Reported status
3763,2015-07-31,Lichtenberg,"Hetze von ""Pro Deutschland"" auf Facebook",Die Kleinstpartei „Pro Deutschland“ veröffentl...,Lichtenberger Register,No,None,None,yes,Unknown,Unknown,Unknown
8212,2017-09-10,Treptow-Köpenick,AfD beim Altglienicker Spätsommerfest,Am 10. September fand das 16. Altglienicker Sp...,Register Treptow-Köpenick,No,52.4211,13.5411,yes,Unknown,Unknown,Unknown
8891,2017-06-30,Lichtenberg,Flüchtlingsfeindliche Beiträge auf NPD-Faceboo...,Die Lichtenberger NPD veröffentlichte auf Face...,Lichtenberger Register,No,None,None,yes,Unknown,Unknown,Unknown


In [284]:
df_map[df_map['Attack'] == 'No']

,Date,District,Header,Story,Source,Attack,latitude,longitude,Propaganda,Structural Discrimination,Material Damage,Reported status
1861,2014-10-17,Mitte,LGBQT feindliche Schmiererei entdeckt!,Am Abend gegen 20.10 Uhr stellen Polizeibeamte...,Polizei Berlin,No,52.5133,13.3761,anonymous,Unknown,yes,yes
3763,2015-07-31,Lichtenberg,"Hetze von ""Pro Deutschland"" auf Facebook",Die Kleinstpartei „Pro Deutschland“ veröffentl...,Lichtenberger Register,No,None,None,identified,Unknown,Unknown,Unknown
4405,2015-03-23,Treptow-Köpenick,Homophobe Vermietungspolitik,Zwei Frauen versuchten bei einer privaten Gart...,Zentrum für Demokratie,No,None,None,Unknown,yes,Unknown,Unknown
5183,2016-10-13,Friedrichshain-Kreuzberg,LGBTIQ*-feindliche Schmiererei am U-Bahnhof Mö...,Am U-Bahnhof Möckernbrücke wurde eine LGBTIQ* ...,Register Charlottenburg-Wilmersdorf,No,52.4991,13.3827,anonymous,Unknown,yes,Unknown
6214,2016-06-03,Reinickendorf,Rassistische und homophobe Schmiererei,Vor der Franz-Marc-Schule in der Treskowallee ...,Register Spandau,No,52.5882,13.2827,anonymous,Unknown,yes,Unknown
6221,2016-06-02,Reinickendorf,Plakate beschmiert und beschädigt,In der ver.di Jugendbildungsstätte Konradshöhe...,ver.di Jugendbildungsstätte Berlin-Konradshöhe...,No,52.586,13.2207,anonymous,Unknown,yes,Unknown
8212,2017-09-10,Treptow-Köpenick,AfD beim Altglienicker Spätsommerfest,Am 10. September fand das 16. Altglienicker Sp...,Register Treptow-Köpenick,No,52.4211,13.5411,identified,Unknown,Unknown,Unknown
8694,2017-07-24,Mitte,Plakat aus LGBTIQ*-feindlichen Gründen zerstört,Im Ortsteil Mitte an der Kreuzung Torstraße Ec...,Berliner Register,No,52.5297,13.401,anonymous,Unknown,yes,Unknown
8891,2017-06-30,Lichtenberg,Flüchtlingsfeindliche Beiträge auf NPD-Faceboo...,Die Lichtenberger NPD veröffentlichte auf Face...,Lichtenberger Register,No,None,None,identified,Unknown,Unknown,Unknown


In [248]:
df_map.loc[df_map['Attack'].isna()==True, 'Attack'] = 'No'

In [253]:
df_map.loc[df_map['Propaganda'].isna()==True, 'Propaganda'] = 'Unknown'

In [257]:
df_map.loc[df_map['Structural Discrimination'].isna()==True, 'Structural Discrimination'] = 'Unknown'

In [259]:
df_map.loc[df_map['Material Damage'].isna()==True, 'Material Damage'] = 'Unknown'

In [261]:
df_map.loc[df_map['Reported status'].isna()==True, 'Reported status'] = 'Unknown'

In [280]:
df_map.loc[df_map['latitude'] == 'Unknown', 'latitude'] = None
df_map.loc[df_map['longitude'] == 'Unknown', 'longitude'] = None

In [563]:
lgbt_index_test = df_lgbt[df_lgbt['Reported status'] == 'yes'].index

In [564]:
map_index_test = df_map[df_map['Reported status'] == 'yes'].index

In [565]:
lgbt_index = df_lgbt.index
map_index = df_map.index

In [566]:
missing_indices = [x for x in list(lgbt_index_test) if x not in map_index_test]
len(missing_indices)

0

In [567]:
really_missing = [x for x in missing_indices if x not in list(map_index)]
len(really_missing)

0

In [568]:
[x for x in missing_indices if x not in really_missing]

[]

In [554]:
df_map.loc[6213,'Story']

'Ein 27-jähriger Mann wurde gegen 22.40 Uhr vor dem S-Bahnhof Herrmannstraße von einem 44-jährigen Mann homophob und rassistisch beleidigt. Der 27-Jährige wollte sich daraufhin entfernen, wurde aber durch den Täter verfolgt. Als er seinen Verfolger offen zu filmen begann, griff der 44-Jährige Täter ihn an und würgte ihn. Zeug_innen intervenierten und die alarmierte Polizei nahm den Täter fest.'

In [555]:
df_lgbt.loc[6213, 'Material Damage'] = np.nan

In [524]:
df_lgbt.loc[3403, 'Material Damage'] = 'yes'

In [522]:
df_lgbt.loc[[7517, 13208], 'Attack']

7517     NaN
13208    NaN
Name: Attack, dtype: object

In [469]:
df_lgbt.loc[missing_indices]

,Date,District,Header,Story,Source,Attack,Propaganda,Structural Discrimination,Material Damage,Reported status
1861,2014-10-17,Mitte,LGBQT feindliche Schmiererei entdeckt!,Am Abend gegen 20.10 Uhr stellen Polizeibeamte...,Polizei Berlin,NaN,NaN,NaN,yes,NaN
3403,2015-09-29,Mitte,Homophobie,Am 29. September 2015 ereignete sich im Ortste...,ReachOut,physical,NaN,NaN,NaN,NaN
5821,2016-07-23,Mitte,Angriff auf schwulen Fahrgast,"Ein Fahrgast, der sich zwischen 17:00 und 18:0...",Polizeimeldung,physical,NaN,NaN,NaN,NaN
6213,2016-06-03,Neukölln,Angegriffen und gewürgt,Ein 27-jähriger Mann wurde gegen 22.40 Uhr vor...,ReachOut,physical,NaN,NaN,NaN,NaN


In [470]:
df_lgbt.loc[missing_indices, 'Material Damage'] = 'yes'

____

## Pickle df_map

In [571]:
# pickle df_map_test
with open('pickles/df_map_test.pickle', 'wb' ) as f:
    pickle.dump(df_map, f)

## Pickle df_lgbt

In [572]:
# pickle df_map_test
with open('pickles/df_lgbt.pickle', 'wb' ) as f:
    pickle.dump(df_lgbt, f)